In [1]:
import pandas as pd

card_infos = pd.read_csv('./shinhan_checkCardInfos.csv')

card_urls = card_infos['card_url'].tolist()
card_names = card_infos['card_name'].tolist()
card_imgs = card_infos['card_img'].tolist()

In [2]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

card_summary = []
card_benefit = []

for url in card_urls:
    url = f'https://www.shinhancard.com/pconts/html/card/apply/check/{url}'

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    driver.implicitly_wait(20)
    driver.get(url)
    time.sleep(3)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 혜택 요약 =============================================
    all_view = soup.find('ul', {'class':'all_view'})
    strong_list = all_view.find_all('strong') # 혜택 제목 리스트
    summary = ''
    for i in range(len(strong_list)):
        summary += f'[{strong_list[i].text}]'
        ul_tag = strong_list[i].find_next('ul', class_='marker_dot')
        li_tags = ul_tag.find_all('li')

        for j in range(len(li_tags)):
            summary += f'{i + 1}. {li_tags[j].text}'
        
    tap_wrap = soup.find('div', {'class':'tab_wrap'})
    hidden_tap = tap_wrap.find_all('li', {'aria-hidden':'true'})

    # 상세 혜택 =============================================    
    benefit_title = [] # 탭 제목들 (혜택 제목)
    benefit = ''
    for i in range(len(hidden_tap)):
        benefit_title.append(hidden_tap[i].find('h2', {'class':'hidden-text'}).text.strip())

    for i in range(len(hidden_tap)):
        benefit += f'###{benefit_title[i]}'
        tit_dep2 = hidden_tap[i].find_all(class_="tit_dep2")

        for title in tit_dep2:
            benefit += f'[{title.text.strip()}]'
            next_sibling = title.find_next_sibling()
            while next_sibling:
                if next_sibling.name == 'div' and 'table_wrap' in next_sibling.get('class', []):
                    table = next_sibling.select_one('table')
                    # 표가 있을 때만 출력
                    if table:
                        rows = table.find_all('tr')
                        benefit += table.find('strong').text.strip()
                        for j, row in enumerate(rows):

                            cells = row.find_all(['th', 'td'])
                            row_data = ['(' + cell.text.strip() + ')' for cell in cells]
                            row_string = ' '.join(row_data)

                            # 각 행을 설명하는 문장 출력
                            sentence = f"표의 {j + 1}번째 행은 {row_string}로 이루어져 있습니다."
                            benefit += sentence
                            
                elif next_sibling.name == 'p':
                    class_list = next_sibling.get('class', [])
                    if ('marker_dot' in class_list) or ('marker_refer' in class_list):
                        benefit += next_sibling.text.strip()
                elif next_sibling.name == 'ul':
                    class_list = next_sibling.get('class', [])
                    if ('marker_dot' in class_list) or ('marker_refer' in class_list) or ('marker_hyphen' in class_list):
                        li_list = next_sibling.find_all('li', recursive=False)
                        for li in li_list:
                            benefit += li.text.strip()

                # 다음 형제 요소 찾을 때 특정 조건을 만족하면 루프 종료
                if 'tit_dep2' in next_sibling.get('class', []) or ('h4' in next_sibling.name and 'tit_dep3' in next_sibling.get('class', [])):
                    break
                
                next_sibling = next_sibling.find_next_sibling()
        
        tit_dep3 = hidden_tap[i].find_all('h4', class_="tit_dep3")
        if tit_dep3:
            for title in tit_dep3:
                benefit += f'[{title.text.strip()}]'
                next_sibling = title.find_next_sibling()
                while next_sibling:
                    if next_sibling.name == 'div' and 'table_wrap' in next_sibling.get('class', []):
                        table = next_sibling.select_one('table')
                        # 표가 있을 때만 출력
                        if table:
                            rows = table.find_all('tr')
                            benefit += table.find('strong').text.strip()
                            for j, row in enumerate(rows):

                                cells = row.find_all(['th', 'td'])
                                row_data = ['(' + cell.text.strip() + ')' for cell in cells]
                                row_string = ' '.join(row_data)

                                # 각 행을 설명하는 문장 출력
                                sentence = f"표의 {j + 1}번째 행은 {row_string}로 이루어져 있습니다."
                                benefit += sentence
                    elif next_sibling.name == 'p':
                        class_list = next_sibling.get('class', [])
                        if ('marker_dot' in class_list) or ('marker_refer' in class_list):
                            benefit += next_sibling.text.strip()
                            
                    elif next_sibling.name == 'ul':
                        class_list = next_sibling.get('class', [])
                        if ('marker_dot' in class_list) or ('marker_refer' in class_list) or ('marker_hyphen' in class_list):
                            li_list = next_sibling.find_all('li', recursive=False)
                            for li in li_list:
                                benefit += li.text.strip()

                    # 다음 형제 요소 찾을 때 특정 조건을 만족하면 루프 종료
                    if 'tit_dep3' in next_sibling.get('class', []):
                        break
                    
                    next_sibling = next_sibling.find_next_sibling()
                    
    card_summary.append(summary)
    card_benefit.append(benefit)
    
driver.quit()

data = {"card_names": card_names, "card_imgs": card_imgs, "card_summarys": card_summary, "card_benefits" : card_benefit}
df = pd.DataFrame(data)

df.to_csv("shinhan_check_benefits.csv", encoding = "utf-8-sig")

In [3]:
shinhan_benefits = pd.read_csv('./shinhan_check_benefits.csv')
shinhan_benefits

,Unnamed: 0,card_names,card_imgs,card_summarys,card_benefits
0,0,신한카드 SOL트래블 체크,https://www.shinhancard.com/pconts/images/cont...,[해외 이용금액 현지 통화 결제 서비스]1. 국제 브랜드 수수료 1%/해외 서비스 ...,###해외 이용금액 현지 통화 결제 서비스[현지통화 결제 서비스란?]신한카드 SOL...
1,1,카카오페이 신한 라이언 체크,https://www.shinhancard.com/pconts/images/cont...,[카카오페이 이용 시 적립]1. 이용금액 출금 기준1. 포인트 적립서비스 적용 기준...,###카카오페이 이용 시 적립[카카오페이 이용금액에 2% 카카오페이포인트 적립 & ...
2,2,카카오페이 신한 춘식이 체크,https://www.shinhancard.com/pconts/images/cont...,[카카오페이 이용 시 적립]1. 이용금액 출금 기준1. 포인트 적립서비스 적용 기준...,###카카오페이 이용 시 적립[카카오페이 이용금액에 2% 카카오페이포인트 적립 & ...
3,3,신한카드 Pick E 체크 캐릭터형(루피),https://www.shinhancard.com/pconts/images/cont...,[4대 편의점 10% 적립]1. Pick 상품 간 중복소지 기준1. 포인트 적립 서...,###4대 편의점 10% 적립[4대 편의점 이용 시 10% 적립]4대 편의점: GS...
4,4,신한카드 Pick I 체크 캐릭터형(쵸파),https://www.shinhancard.com/pconts/images/cont...,[4대 편의점 10% 적립]1. Pick 상품 간 중복소지 기준1. 포인트 적립 서...,###4대 편의점 10% 적립[4대 편의점 이용 시 10% 적립]4대 편의점: GS...
...,...,...,...,...,...
98,98,부산가족사랑 신한카드 S-Choice 체크,https://www.shinhancard.com/pconts/images/cont...,[선택형 할인]1. 부산가족사랑 신한카드 S-Choice 체크의 선택형 서비스(택1...,###선택형 할인[부산가족사랑 신한 S-Choice 체크의 할인 서비스(3가지 서비...
99,99,신한카드 On 체크(한국유치원총연합회 교원증),https://www.shinhancard.com/pconts/images/cont...,[간편결제 이용시 적립(On 체크)]1. 간편결제(Pay) 이용 시 최대 2% 적립...,###간편결제 이용시 적립(On 체크)[간편결제(Pay) 이용 시 최대 2% 적립]...
100,100,삼성증권 CMA 신한카드 Big Plus 체크,https://www.shinhancard.com/pconts/images/cont...,[삼성증권 CMA 서비스]1. 최대 6만원 리워드 혜택(조건충족 시)1. 365일 ...,###삼성증권 CMA 서비스[알찬 투자수익은 기본(RP및 MMF로 운용)]최대 6만...
101,101,신한카드 Hey Young 체크(청소년연맹 대학생 교사),https://www.shinhancard.com/pconts/images/cont...,"[생활 캐시백]1. 대중교통, 이동통신, 영화, 편의점, 커피, 쿠팡\n ...",###생활 캐시백[대중교통 이용금액 5% 캐시백]마을/시내/시외/공항버스/지하철(고...
